In [15]:
import numpy as np
import pandas as pd





In [16]:
FILE_PATH = "famabliss_strips_2025-11-28.xlsx"
SHEET = "prices"



In [17]:

prices = pd.read_excel(FILE_PATH, sheet_name=SHEET)
prices["date"] = pd.to_datetime(prices["date"])
prices = prices.sort_values("date").reset_index(drop=True)

nov = prices[(prices["date"].dt.month == 11) & (prices["date"].dt.year.between(2020, 2025))].copy()
nov = nov.set_index(nov["date"].dt.year).sort_index()


DF = nov[[1, 2, 3, 4, 5]].copy()

print("November discount factors (prices per $1 face):")
print(nov[["date", 1, 2, 3, 4, 5]])



November discount factors (prices per $1 face):
           date         1         2         3         4         5
date                                                             
2020 2020-11-30  0.998839  0.997047  0.994464  0.988809  0.981515
2021 2021-11-30  0.997523  0.988614  0.974796  0.958322  0.943772
2022 2022-11-30  0.954375  0.918220  0.887608  0.857356  0.830593
2023 2023-11-30  0.951080  0.911951  0.876957  0.842191  0.809378
2024 2024-11-29  0.959187  0.920923  0.885440  0.850418  0.817795
2025 2025-11-28  0.964936  0.932866  0.900919  0.868240  0.836057


In [ ]:

def irr(cashflows):

    try:
        import numpy_financial as nf
        return float(nf.irr(cashflows))
    except Exception:
        cfs = np.array(cashflows, dtype=float)

        def npv(r):
            return np.sum(cfs / ((1 + r) ** np.arange(len(cfs))))

        lo, hi = -0.9999, 10.0
        f_lo, f_hi = npv(lo), npv(hi)
        if f_lo * f_hi > 0:
            return np.nan

        for _ in range(250):
            mid = (lo + hi) / 2
            f_mid = npv(mid)
            if f_lo * f_mid <= 0:
                hi, f_hi = mid, f_mid
            else:
                lo, f_lo = mid, f_mid
        return float((lo + hi) / 2)

def spot_yield_from_df(df_n, n):
    return (df_n ** (-1.0 / n)) - 1.0



In [19]:

MV = 100_000_000.0
haircut = 0.02
initial_capital = 2 * haircut * MV  

years = [2020, 2021, 2022, 2023, 2024, 2025]

df5_2020 = float(DF.loc[2020, 5])
long_face = MV / df5_2020  

long_vals = {}
for y in years:
    rem = 2025 - y  
    price = 1.0 if rem == 0 else float(DF.loc[y, rem])
    long_vals[y] = long_face * price


short_face = {y: MV / float(DF.loc[y, 1]) for y in [2020, 2021, 2022, 2023]}

pnl_rows = []
for y in [2020, 2021, 2022, 2023, 2024]:
    long_pnl = long_vals[y + 1] - long_vals[y]
    short_pnl = -(short_face[y] - MV) if y <= 2023 else 0.0
    pnl_rows.append([f"Nov {y}→Nov {y+1}", long_pnl, short_pnl, long_pnl + short_pnl])

pnl_df = pd.DataFrame(pnl_rows, columns=["Period", "Long P&L", "Short P&L", "Total P&L"])
print("\n1.1 Year-by-year P&L (actual, $m):")
print((pnl_df.assign(**{c: pnl_df[c] / 1e6 for c in ["Long P&L", "Short P&L", "Total P&L"]})
      .rename(columns={"Long P&L": "Long P&L ($m)", "Short P&L": "Short P&L ($m)", "Total P&L": "Total P&L ($m)"})))


inj = {
    2021: short_face[2020] - MV,
    2022: short_face[2021] - MV,
    2023: short_face[2022] - MV,
    2024: short_face[2023],  
}
print("\nCash injections required at rollover (actual):")
for k, v in inj.items():
    print(k, f"${v:,.2f}  ({v/1e6:.6f}m)")


cashflows = {
    2021: -(short_face[2020] - MV),
    2022: -(short_face[2021] - MV),
    2023: -(short_face[2022] - MV),
    2024: -short_face[2023],
    2025: +long_face
}
net_profit = sum(cashflows.values())

cf_actual = [
    -initial_capital,
    cashflows[2021],
    cashflows[2022],
    cashflows[2023],
    cashflows[2024],
    cashflows[2025] + initial_capital
]

irr_actual = irr(cf_actual)

print("\nNet profit by Nov 2025 (actual):", f"${net_profit:,.2f}")
print("Return on initial $4m (profit / 4m):", net_profit / initial_capital)
print("IRR (annual, incl. all injections and returning $4m at end):", irr_actual)




1.1 Year-by-year P&L (actual, $m):
              Period  Long P&L ($m)  Short P&L ($m)  Total P&L ($m)
0  Nov 2020→Nov 2021      -2.362973       -0.116228       -2.479200
1  Nov 2021→Nov 2022      -7.204655       -0.248347       -7.453003
2  Nov 2022→Nov 2023       2.480175       -4.780616       -2.300442
3  Nov 2023→Nov 2024       4.812566       -5.143642       -0.331076
4  Nov 2024→Nov 2025       4.158168        0.000000        4.158168

Cash injections required at rollover (actual):
2021 $116,227.51  (0.116228m)
2022 $248,347.30  (0.248347m)
2023 $4,780,616.14  (4.780616m)
2024 $105,143,642.22  (105.143642m)

Net profit by Nov 2025 (actual): $-8,405,552.01
Return on initial $4m (profit / 4m): -2.101388003418863
IRR (annual, incl. all injections and returning $4m at end): -0.06312646283020623


In [20]:

d = {0: 1.0}
for n in range(1, 6):
    d[n] = float(DF.loc[2020, n])


fd = {t: d[t + 1] / d[t] for t in range(0, 5)}
fwd_1y_rates = {t: (1.0 / fd[t]) - 1.0 for t in range(0, 5)}  

print("\n1.2 Forward-implied 1Y spot rates from Nov 2020 curve:")
for t in range(0, 5):
    print(f"Nov {2020+t}→Nov {2020+t+1}: {100*fwd_1y_rates[t]:.4f}%")


long_face_h = MV / d[5]

long_vals_h = {t: long_face_h * (d[5] / d[t]) for t in range(0, 5)}
long_vals_h[5] = long_face_h  


short_face_h = {t: MV / fd[t] for t in range(0, 4)}  


pnl_h_rows = []
for t in range(0, 5):
    long_pnl = long_vals_h[t + 1] - long_vals_h[t]
    short_pnl = -(short_face_h[t] - MV) if t <= 3 else 0.0
    pnl_h_rows.append([f"Nov {2020+t}→Nov {2020+t+1}", long_pnl, short_pnl, long_pnl + short_pnl])

pnl_h = pd.DataFrame(pnl_h_rows, columns=["Period", "Long P&L", "Short P&L", "Total P&L"])
print("\n1.2 Year-by-year P&L (hypothetical, $m):")
print((pnl_h.assign(**{c: pnl_h[c] / 1e6 for c in ["Long P&L", "Short P&L", "Total P&L"]})
      .rename(columns={"Long P&L": "Long P&L ($m)", "Short P&L": "Short P&L ($m)", "Total P&L": "Total P&L ($m)"})))

cashflows_h = {
    2021: -(short_face_h[0] - MV),
    2022: -(short_face_h[1] - MV),
    2023: -(short_face_h[2] - MV),
    2024: -short_face_h[3],
    2025: +long_face_h
}
net_profit_h = sum(cashflows_h.values())

cf_h = [
    -initial_capital,
    cashflows_h[2021],
    cashflows_h[2022],
    cashflows_h[2023],
    cashflows_h[2024],
    cashflows_h[2025] + initial_capital
]
irr_h = irr(cf_h)

print("\nNet profit by Nov 2025 (hypothetical):", f"${net_profit_h:,.2f}")
print("IRR (annual, hypothetical):", irr_h)




1.2 Forward-implied 1Y spot rates from Nov 2020 curve:
Nov 2020→Nov 2021: 0.1162%
Nov 2021→Nov 2022: 0.1797%
Nov 2022→Nov 2023: 0.2597%
Nov 2023→Nov 2024: 0.5719%
Nov 2024→Nov 2025: 0.7431%

1.2 Year-by-year P&L (hypothetical, $m):
              Period  Long P&L ($m)  Short P&L ($m)  Total P&L ($m)
0  Nov 2020→Nov 2021       0.116228       -0.116228        0.000000
1  Nov 2021→Nov 2022       0.179950       -0.179741        0.000209
2  Nov 2022→Nov 2023       0.260518       -0.259748        0.000769
3  Nov 2023→Nov 2024       0.575070       -0.571886        0.003184
4  Nov 2024→Nov 2025       0.751516        0.000000        0.751516

Net profit by Nov 2025 (hypothetical): $755,678.03
IRR (annual, hypothetical): 0.00617619352725296


In [24]:

print("\n 1.3 Curve slope comparison (spot yields from DFs):")
for y in [2020, 2025]:
    y1 = spot_yield_from_df(float(DF.loc[y, 1]), 1)
    y5 = spot_yield_from_df(float(DF.loc[y, 5]), 5)
    print(f"{y}: y1={100*y1:.4f}%, y5={100*y5:.4f}%, spread(5y-1y)={100*(y5-y1):.4f}%")



 1.3 Curve slope comparison (spot yields from DFs):
2020: y1=0.1162%, y5=0.3739%, spread(5y-1y)=0.2576%
2025: y1=3.6339%, y5=3.6461%, spread(5y-1y)=0.0122%
